In [40]:
#from setup import *
#from agents import *
#from env import *
gameTypes, numBuyers, numSellers, numTokens, numRounds, numPeriods, numSteps = '1001', 2, 6, 3, 1, 100, 9
disclosure = ['rnd', 'period', 'step','currentBid','currentAsk','buy','sell','price','sale']
buyerStrategies = ['PPO', 'TruthTeller']
sellerStrategies = ['TruthTeller', 'TruthTeller','TruthTeller', 'TruthTeller','TruthTeller', 'TruthTeller']
metaData = [gameTypes, numBuyers, numSellers, numTokens, numRounds, numPeriods, numSteps, disclosure, buyerStrategies, sellerStrategies]
gameData = metaData[0:7]
gameTypes, numBuyers, numSellers, numTokens, numRounds, numPeriods, numSteps = gameData
disclosure, buyerStrategies, sellerStrategies = metaData[7:]
buyers, sellers = generateAgents(gameData,buyerStrategies,sellerStrategies,disclosure)
log = Log(gameData, buyerStrategies, sellerStrategies, disclosure)

for rnd in range(numRounds):
    roundData = roundSetup(*gameData)
    redemptionValues, tokenCosts, demand, supply, prices, peq, qeq  = roundData[0:7]
    [buyerReservationPrices, sellerReservationPrices, buyerSurplus, sellerSurplus, totalSurplus, buyerSurplusFrac, sellerSurplusFrac] = roundData[7:]
    log.addRound([rnd] + roundData)
    resetRounds(buyers, sellers, redemptionValues, tokenCosts)
    for period in range(numPeriods):
        resetPeriods(buyers, sellers)
        for step in range(numSteps):
            resetSteps(buyers, sellers)
            bids, asks = collectOffers(buyers, sellers)
            currentAsk, currentAskIdx, currentBid, currentBidIdx = bestOffers(bids, asks)
            price, buy, sell = trade(buyers, sellers, currentAsk, currentAskIdx, currentBid, currentBidIdx)
            bprofit, sprofit = 0, 0
            if price > 0:
                buyers[currentBidIdx].transact(price)
                sellers[currentAskIdx].transact(price)
                bprofit = buyers[currentBidIdx].stepProfits
                sprofit = sellers[currentAskIdx].stepProfits
            log.addStep([rnd, period, step, bids, asks, currentBid, currentBidIdx, currentAsk, currentAskIdx, buy, sell, price, price>0, bprofit, sprofit])
            observe(buyers, sellers, log.disclose()) # observe new state
            updateStates(buyers, sellers)
        updatePolicy(buyers, sellers)
        
        if period % (numPeriods/numPeriods) == 0:
            print(f'\n{period}')
            display(log.getPeriod(rnd, period).tail(100)[['bprofit', 'currentBidIdx']].groupby('currentBidIdx').sum())
            display(log.getPeriod(rnd, period).tail(100)[['sprofit', 'currentAskIdx']].groupby('currentAskIdx').sum())
            display(log.getPeriod(rnd,period))

tensor([0.4714], grad_fn=<SigmoidBackward0>) tensor([0.2015], grad_fn=<MulBackward0>)
tensor([0.3091])
0.30911093950271606 

tensor([0.9433], grad_fn=<SigmoidBackward0>) tensor([1.3978e-05], grad_fn=<MulBackward0>)
tensor([0.9433])
0.9433107376098633 

tensor([0.9790], grad_fn=<SigmoidBackward0>) tensor([4.2085e-06], grad_fn=<MulBackward0>)
tensor([0.9790])
0.9790283441543579 

tensor([0.9713], grad_fn=<SigmoidBackward0>) tensor([1.4357e-06], grad_fn=<MulBackward0>)
tensor([0.9713])
0.971271276473999 


0


,bprofit
currentBidIdx,
0.0,-25.65
1.0,95.75


,sprofit
currentAskIdx,
0,19.25
1,26.50
2,90.15
4,89.80


,rnd,period,step,bids,asks,currentBid,currentBidIdx,currentAsk,currentAskIdx,buy,sell,price,sale,bprofit,sprofit
0,0,0,0,"[65.5, 100.0]","[23.3, 19.9, 7.9, 34.4, 0.0, 49.0]",100.0,1.0,0.0,4,1.0,1.0,50.00,True,50.00,50.00
1,0,0,1,"[97.2, 72.9]","[23.3, 19.9, 7.9, 34.4, 19.0, 49.0]",97.2,0.0,7.9,2,1.0,1.0,52.55,True,6.75,44.65
2,0,0,2,"[99.0, 72.9]","[23.3, 19.9, 8.0, 34.4, 19.0, 49.0]",99.0,0.0,8.0,2,1.0,1.0,53.50,True,-9.70,45.50
3,0,0,3,"[98.6, 72.9]","[23.3, 19.9, 58.8, 34.4, 19.0, 49.0]",98.6,0.0,19.0,4,1.0,1.0,58.80,True,-22.70,39.80
4,0,0,4,"[nan, 72.9]","[23.3, 19.9, 58.8, 34.4, 56.2, 49.0]",72.9,1.0,19.9,1,1.0,1.0,46.40,True,26.50,26.50
5,0,0,5,"[nan, 61.8]","[23.3, 28.1, 58.8, 34.4, 56.2, 49.0]",61.8,1.0,23.3,0,1.0,1.0,42.55,True,19.25,19.25
6,0,0,6,"[nan, nan]","[41.1, 28.1, 58.8, 34.4, 56.2, 49.0]",NaN,NaN,28.1,1,NaN,NaN,NaN,False,0.00,0.00
7,0,0,7,"[nan, nan]","[41.1, 28.1, 58.8, 34.4, 56.2, 49.0]",NaN,NaN,28.1,1,NaN,NaN,NaN,False,0.00,0.00
8,0,0,8,"[nan, nan]","[41.1, 28.1, 58.8, 34.4, 56.2, 49.0]",NaN,NaN,28.1,1,NaN,NaN,NaN,False,0.00,0.00


tensor([0.8513], grad_fn=<SigmoidBackward0>) tensor([0.0503], grad_fn=<MulBackward0>)
tensor([0.7997])
0.7996672987937927 

tensor([0.9891], grad_fn=<SigmoidBackward0>) tensor([3.0621e-06], grad_fn=<MulBackward0>)
tensor([0.9891])
0.9890791177749634 

tensor([0.9806], grad_fn=<SigmoidBackward0>) tensor([3.0521e-06], grad_fn=<MulBackward0>)
tensor([0.9806])
0.9806203842163086 

tensor([0.9740], grad_fn=<SigmoidBackward0>) tensor([1.5274e-06], grad_fn=<MulBackward0>)
tensor([0.9740])
0.9739600419998169 


1


...

,rnd,period,step,bids,asks,currentBid,currentBidIdx,currentAsk,currentAskIdx,buy,sell,price,sale,bprofit,sprofit
882,0,98,0,"[94.5, 100.0]","[23.3, 19.9, 7.9, 34.4, 0.0, 49.0]",100.0,1.0,0.0,4,True,True,50.00,True,50.00,50.00
883,0,98,1,"[99.6, 72.9]","[23.3, 19.9, 7.9, 34.4, 19.0, 49.0]",99.6,0.0,7.9,2,True,True,53.75,True,5.55,45.85
884,0,98,2,"[99.0, 72.9]","[23.3, 19.9, 8.0, 34.4, 19.0, 49.0]",99.0,0.0,8.0,2,True,True,53.50,True,-9.70,45.50
885,0,98,3,"[98.7, 72.9]","[23.3, 19.9, 58.8, 34.4, 19.0, 49.0]",98.7,0.0,19.0,4,True,True,58.85,True,-22.75,39.85
886,0,98,4,"[nan, 72.9]","[23.3, 19.9, 58.8, 34.4, 56.2, 49.0]",72.9,1.0,19.9,1,True,True,46.40,True,26.50,26.50
887,0,98,5,"[nan, 61.8]","[23.3, 28.1, 58.8, 34.4, 56.2, 49.0]",61.8,1.0,23.3,0,True,True,42.55,True,19.25,19.25
888,0,98,6,"[nan, nan]","[41.1, 28.1, 58.8, 34.4, 56.2, 49.0]",NaN,NaN,28.1,1,NaN,NaN,NaN,False,0.00,0.00
889,0,98,7,"[nan, nan]","[41.1, 28.1, 58.8, 34.4, 56.2, 49.0]",NaN,NaN,28.1,1,NaN,NaN,NaN,False,0.00,0.00
890,0,98,8,"[nan, nan]","[41.1, 28.1, 58.8, 34.4, 56.2, 49.0]",NaN,NaN,28.1,1,NaN,NaN,NaN,False,0.00,0.00


tensor([0.8650], grad_fn=<SigmoidBackward0>) tensor([0.0503], grad_fn=<MulBackward0>)
tensor([0.7930])
0.7930301427841187 

tensor([0.9889], grad_fn=<SigmoidBackward0>) tensor([3.1050e-06], grad_fn=<MulBackward0>)
tensor([0.9889])
0.9888903498649597 

tensor([0.9806], grad_fn=<SigmoidBackward0>) tensor([3.0951e-06], grad_fn=<MulBackward0>)
tensor([0.9806])
0.980560302734375 

tensor([0.9738], grad_fn=<SigmoidBackward0>) tensor([1.5233e-06], grad_fn=<MulBackward0>)
tensor([0.9739])
0.9738514423370361 


99


,bprofit
currentBidIdx,
0.0,-26.80
1.0,95.75


,sprofit
currentAskIdx,
0,19.25
1,26.50
2,91.25
4,89.85


,rnd,period,step,bids,asks,currentBid,currentBidIdx,currentAsk,currentAskIdx,buy,sell,price,sale,bprofit,sprofit
891,0,99,0,"[89.7, 100.0]","[23.3, 19.9, 7.9, 34.4, 0.0, 49.0]",100.0,1.0,0.0,4,True,True,50.00,True,50.00,50.00
892,0,99,1,"[99.4, 72.9]","[23.3, 19.9, 7.9, 34.4, 19.0, 49.0]",99.4,0.0,7.9,2,True,True,53.65,True,5.65,45.75
893,0,99,2,"[99.0, 72.9]","[23.3, 19.9, 8.0, 34.4, 19.0, 49.0]",99.0,0.0,8.0,2,True,True,53.50,True,-9.70,45.50
894,0,99,3,"[98.7, 72.9]","[23.3, 19.9, 58.8, 34.4, 19.0, 49.0]",98.7,0.0,19.0,4,True,True,58.85,True,-22.75,39.85
895,0,99,4,"[nan, 72.9]","[23.3, 19.9, 58.8, 34.4, 56.2, 49.0]",72.9,1.0,19.9,1,True,True,46.40,True,26.50,26.50
896,0,99,5,"[nan, 61.8]","[23.3, 28.1, 58.8, 34.4, 56.2, 49.0]",61.8,1.0,23.3,0,True,True,42.55,True,19.25,19.25
897,0,99,6,"[nan, nan]","[41.1, 28.1, 58.8, 34.4, 56.2, 49.0]",NaN,NaN,28.1,1,NaN,NaN,NaN,False,0.00,0.00
898,0,99,7,"[nan, nan]","[41.1, 28.1, 58.8, 34.4, 56.2, 49.0]",NaN,NaN,28.1,1,NaN,NaN,NaN,False,0.00,0.00
899,0,99,8,"[nan, nan]","[41.1, 28.1, 58.8, 34.4, 56.2, 49.0]",NaN,NaN,28.1,1,NaN,NaN,NaN,False,0.00,0.00


In [33]:
class PPO(Trader):
    def __init__(self, gameData, disclosure, index, buyer, type):
        super().__init__(gameData, disclosure, index, buyer, type)
        self.learningRate = 0.0002
        self.gamma = 0.99
        self.numActions = 100
        self.depth = 1
        self.numStates = 8 + 6 * self.depth
        agentState = [0, -1, -1, -1, -1, -1, -1, -1]
        history = [-1, -1, -1, -1, -1, -1] * self.depth
        self.state = np.nan_to_num(np.array(agentState + history, dtype = np.float32), nan=-9)
        self.model = PPOModel(self.numStates)
        self.rollout = []
        self.done = False

    def observe(self):
        agentState = [self.periodStep, self.stepTokenValue, self.stepBid, self.stepAsk, self.stepTrades, self.stepProfits, self.periodTrades, self.periodProfits]
        history = self.df.iloc[-self.depth:][['currentBid', 'currentAsk', 'buy', 'sell', 'price', 'price']].values.reshape(-1,).tolist()
        if len(history) != (6 * self.depth):
            history = [-1] * (6 * self.depth)
        self.newState = np.nan_to_num(np.array(agentState + history, dtype = np.float32), nan=-9)
        if self.periodStep == self.numSteps:
            self.done = True
        self.rollout.append((self.state, self.action, self.stepProfits/100.0, self.newState, self.log_prob.item(), self.done))
        self.state = self.newState
        if self.periodStep == self.numSteps:
            self.model.put_data(self.rollout)
            self.rollout = []  
        
    def bid(self):
        self.stepBid = np.nan
        if self.stepTokenValue > 0:
            mu, std = self.model.pi(torch.from_numpy(self.state).float())
            print(mu, std)
            dist = Normal(mu, std)
            self.action = dist.sample()
            print(self.action)
            self.log_prob = dist.log_prob(self.action)
            self.action = np.clip(self.action, -1, 1)
            print(self.action.item(), '\n')
            frac = (self.action.item()+1)/2
            self.stepBid = np.round(frac * 100,1)
        return self.stepBid
        
    def ask(self):
        self.stepAsk = np.nan
        if self.stepTokenValue > 0:
            self.mu, self.std = self.model.pi(torch.from_numpy(self.state).float())
            dist = Normal(self.mu, self.std)
            self.action = dist.sample()
            self.log_prob = dist.log_prob(self.action)
            frac = self.action.item()
            self.stepAsk = np.round(frac * 100,1)
        return self.stepAsk

def updatePolicy(buyers, sellers):
    for buyer in buyers:
        if buyer.type == 2:
            buyer.model.train_net()
    for seller in sellers:
        if seller.type == 2:
            seller.model.train_net()

def updateStates(buyers, sellers):
    for buyer in buyers:
        if buyer.type == 2:
            buyer.observe()
    for seller in sellers:
        if seller.type == 2:
            seller.observe()

In [39]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal
from torch.distributions import Beta

learning_rate  = 0.0003
gamma          = 0.9
lmbda          = 0.9
eps_clip       = 0.2
K_epoch        = 10
rollout_len    = 3
buffer_size    = 10
minibatch_size = 32

class PPOModel(nn.Module):
    def __init__(self, numStates):
        super(PPOModel, self).__init__()
        self.numStates = numStates
        self.data = []
        self.fc1   = nn.Linear(self.numStates,128)
        self.fc_mu = nn.Linear(128,1)
        self.fc_std  = nn.Linear(128,1)
        self.fc_v = nn.Linear(128,1)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.optimization_step = 0

    def pi(self, x, softmax_dim = 0):
        x = F.relu(self.fc1(x))
        mu = torch.sigmoid(self.fc_mu(x))
        std = 0.4*F.sigmoid(self.fc_std(x))
        return mu, std
    
    def v(self, x):
        x = F.relu(self.fc1(x))
        v = self.fc_v(x)
        return v
      
    def put_data(self, transition):
        self.data.append(transition)
        
    def make_batch(self):
        s_batch, a_batch, r_batch, s_prime_batch, prob_a_batch, done_batch = [], [], [], [], [], []
        data = []
        for j in range(buffer_size):
            for i in range(minibatch_size):
                rollout = self.data.pop()
                s_lst, a_lst, r_lst, s_prime_lst, prob_a_lst, done_lst = [], [], [], [], [], []
                for transition in rollout:
                    s, a, r, s_prime, prob_a, done = transition      
                    s_lst.append(s)
                    a_lst.append([a])
                    r_lst.append([r])
                    s_prime_lst.append(s_prime)
                    prob_a_lst.append([prob_a])
                    done_mask = 0 if done else 1
                    done_lst.append([done_mask])
                s_batch.append(s_lst)
                a_batch.append(a_lst)
                r_batch.append(r_lst)
                s_prime_batch.append(s_prime_lst)
                prob_a_batch.append(prob_a_lst)
                done_batch.append(done_lst)
            mini_batch = torch.tensor(s_batch, dtype=torch.float), torch.tensor(a_batch, dtype=torch.float), \
                          torch.tensor(r_batch, dtype=torch.float), torch.tensor(s_prime_batch, dtype=torch.float), \
                          torch.tensor(done_batch, dtype=torch.float), torch.tensor(prob_a_batch, dtype=torch.float)
            data.append(mini_batch)
        return data

    def calc_advantage(self, data):
        data_with_adv = []
        for mini_batch in data:
            s, a, r, s_prime, done_mask, old_log_prob = mini_batch
            with torch.no_grad():
                td_target = r + gamma * self.v(s_prime) * done_mask
                delta = td_target - self.v(s)
            delta = delta.numpy()
            advantage_lst = []
            advantage = 0.0
            for delta_t in delta[::-1]:
                advantage = gamma * lmbda * advantage + delta_t[0]
                advantage_lst.append([advantage])
            advantage_lst.reverse()
            advantage = torch.tensor(advantage_lst, dtype=torch.float)
            data_with_adv.append((s, a, r, s_prime, done_mask, old_log_prob, td_target, advantage))
        return data_with_adv

    def train_net(self):
        if len(self.data) == minibatch_size * buffer_size:
            data = self.make_batch()
            data = self.calc_advantage(data)
            for i in range(K_epoch):
                for mini_batch in data:
                    s, a, r, s_prime, done_mask, old_log_prob, td_target, advantage = mini_batch
                    mu, std = self.pi(s, softmax_dim=1)
                    dist = Normal(mu, std)
                    log_prob = dist.log_prob(a)
                    ratio = torch.exp(log_prob - old_log_prob)  # a/b == exp(log(a)-log(b))
                    surr1 = ratio * advantage
                    surr2 = torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage
                    loss = -torch.min(surr1, surr2) + F.smooth_l1_loss(self.v(s) , td_target)
                    self.optimizer.zero_grad()
                    loss.mean().backward()
                    nn.utils.clip_grad_norm_(self.parameters(), 1.0)
                    self.optimizer.step()
                    self.optimization_step += 1


In [14]:
def generateAgents(gameData,buyerStrategies,sellerStrategies,disclosure):
    buyers, sellers = [], []
    for idx,i in enumerate(buyerStrategies):
        if i == 'TruthTeller':
            buyers.append(TruthTeller(gameData, disclosure, index=idx, buyer=0, type=0)) 
        if i == 'ZeroIntelligence':
            buyers.append(ZeroIntelligence(gameData, disclosure, index=idx, buyer=0, type=1)) 
        if i == 'REINFORCE':
            buyers.append(REINFORCE(gameData, disclosure, index=idx, buyer=0, type=2)) 
        if i == 'PPO':
            buyers.append(PPO(gameData, disclosure, index=idx, buyer=0, type=2)) 
        if i == 'SAC':
            buyers.append(SAC(gameData, disclosure, index=idx, buyer=0, type=2)) 
        if i == 'DQN':
            buyers.append(DQN(gameData, disclosure, index=idx, buyer=0, type=2)) 

    for idx,i in enumerate(sellerStrategies):
        if i == 'TruthTeller':
            sellers.append(TruthTeller(gameData, disclosure, index=idx, buyer=1, type=0)) 
        if i == 'ZeroIntelligence':
            sellers.append(ZeroIntelligence(gameData, disclosure, index=idx, buyer=1, type=1)) 
        if i == 'REINFORCE':
            sellers.append(REINFORCE(gameData, disclosure, index=idx, buyer=1, type=2)) 
        if i == 'PPO':
            sellers.append(PPO(gameData, disclosure, index=idx, buyer=1, type=2)) 
        if i == 'SAC':
            sellers.append(SAC(gameData, disclosure, index=idx, buyer=1, type=2)) 
        if i == 'DQN':
            sellers.append(DQN(gameData, disclosure, index=idx, buyer=1, type=2)) 
    return buyers, sellers